In [1]:
# script to save the rankings for the mechanism
import os
import sys
import copy
import pickle
import pandas as pd

import numpy as np
import rmgpy.chemkin
import cantera as ct

import matplotlib.pyplot as plt
%matplotlib inline

## Load the model for reaction and species descriptions

In [2]:
basedir = '/work/westgroup/harris.se/autoscience/reaction_calculator/delay_uncertainty/base_rmg_1week'

cantera_file = os.path.join(basedir, 'chem_annotated.cti')
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')

species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport, use_chemkin_names=True)

gas = ct.Solution(cantera_file)
perturbed_cti_path = os.path.join(basedir, 'perturbed.cti')
perturbed_gas = ct.Solution(perturbed_cti_path)

# This cti -> rmg converter dictionary can be made using rmg_tools/ct2rmg_dict.py
with open(os.path.join(basedir, 'ct2rmg_rxn.pickle'), 'rb') as handle:
    ct2rmg_rxn = pickle.load(handle)

print(f'{len(species_list)} species loaded')
print(f'{len(reaction_list)} reactions loaded')

For species Ar, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species He, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species butane(1), discontinuity in h/RT detected at Tmid = 1389.0
	Value computed using low-temperature polynomial:  6.672352745666894
	Value computed using high-temperature polynomial: 6.314788936675075

For species CH(3), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  81.27192833333334
	Value computed using high-temperature polynomial: 81.30542726833333

For species C2H(4), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  79.04515833333333
	Value computed using high-temperature 

130 species loaded
2488 reactions loaded


In [9]:
N = len(gas.species())
M = len(gas.reactions())

## Gather Uncertainty Rankings

In [3]:
# rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/reaction_uncertainty.npy'
# sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/species_uncertainty.npy'

rxn_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_reaction_uncertainty.npy'
sp_uncertainty_file = '/work/westgroup/harris.se/autoscience/reaction_calculator/models/base_rmg_1week/gao_species_uncertainty.npy'

rmg_rxn_uncertainty = np.load(rxn_uncertainty_file)
rmg_sp_uncertainty = np.load(sp_uncertainty_file)

assert len(rmg_rxn_uncertainty) == len(reaction_list)
assert len(rmg_sp_uncertainty) == len(species_list)


rxn_uncertainty = np.zeros(len(gas.reactions()))
for ct_index in range(len(rxn_uncertainty)):
    rxn_uncertainty[ct_index] = rmg_rxn_uncertainty[ct2rmg_rxn[ct_index]]

# Cantera species should be in same rmg order, but this makes sure for us
for i in range(len(species_list)):
    assert str(species_list[i]) == gas.species_names[i]

sp_uncertainty = rmg_sp_uncertainty

## Gather Sensitivity Rankings

In [110]:
# load the giant base delays matrix
base_delay_file = os.path.join(basedir, 'total_base_delays.npy')
base_delays = np.load(base_delay_file)

# Load the giant delays matrix
total_delay_file = os.path.join(basedir, 'total_perturbed_mech_delays.npy')
total_delays = np.load(total_delay_file)

assert total_delays.shape[1] == len(base_delays)

total_base_delays = np.repeat(np.matrix(base_delays), total_delays.shape[0], axis=0)
assert total_base_delays.shape == total_delays.shape

total_delays[total_delays == 0] = np.nan


In [111]:
d_ln_tau = np.log(total_delays) - np.log(total_base_delays)

/work/westgroup/harris.se/tst_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [6]:
phi_dicts = []

for table_index in range(1, 13):
    
    # Load the experimental conditions
    ignition_delay_data = '/work/westgroup/harris.se/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    # ignition_delay_data = '/home/moon/autoscience/autoscience/butane/experimental_data/butane_ignition_delay.csv'
    df_exp = pd.read_csv(ignition_delay_data)
    table_exp = df_exp[df_exp['Table'] == table_index]
    # Define Initial conditions using experimental data
    tau_exp = table_exp['time (ms)'].values.astype(float)  # ignition delay
    T7 = table_exp['T_C'].values  # Temperatures
    P7 = table_exp['nominal pressure(atm)'].values * ct.one_atm  # pressures in atm
    phi7 = table_exp['phi'].values  # equivalence ratios
    # list of starting conditions
    # Mixture compositions taken from table 2 of
    # https://doi-org.ezproxy.neu.edu/10.1016/j.combustflame.2010.01.016
    concentrations = []
    # for phi = 1
    x_diluent = 0.7649
    conc_dict = {
        'O2(2)': 0.2038,
        'butane(1)': 0.03135
    }

    
    x_N2 = table_exp['%N2'].values[0] / 100.0 * x_diluent
    x_Ar = table_exp['%Ar'].values[0] / 100.0 * x_diluent
    x_CO2 = table_exp['%CO2'].values[0] / 100.0 * x_diluent
    conc_dict['N2'] = x_N2
    conc_dict['Ar'] = x_Ar
    conc_dict['CO2(7)'] = x_CO2
    
    phi_dicts.append(conc_dict)
        
        

In [7]:
# There are 12 * K different simulation settings. We need each parameter estimate at each setting
# Create a matrix with temperatures and one with pressures
T = np.linspace(663, 1077, 51)
table_temperatures = np.repeat(np.matrix(T), 12, axis=1)
temperatures = np.repeat(table_temperatures, total_delays.shape[0], axis=0)

pressures = np.zeros(temperatures.shape)
for i in range(pressures.shape[1]):
    if int(i / 51) in [0, 3, 6, 9]:
        pressures[:, i] = 10.0 * 101325.0
    elif int(i / 51) in [1, 4, 7, 10]:
        pressures[:, i] = 20.0 * 101325.0
    elif int(i / 51) in [2, 5, 8, 11]:
        pressures[:, i] = 30.0 * 101325.0

        
phis = np.zeros(temperatures.shape)
for i in range(phis.shape[1]):
    if int(i / 153) == 0:
        phis[:, i] = 0.3
    elif int(i / 153) == 1:
        phis[:, i] = 0.5
    elif int(i / 153) == 2:
        phis[:, i] = 1.0
    elif int(i / 153) == 3:
        phis[:, i] = 2.0

# P = np.array([10, 20, 30, 10, 20, 30, 10, 20, 30, 10, 20, 30]) * 101325.0
# pressures = np.repeat(np.matrix(P), total_delays.shape[0], axis=0)

In [10]:
G_base = np.zeros((N, total_delays.shape[1]))
G_perturbed = np.zeros((N, total_delays.shape[1]))


# get base G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_base[j, i] = gas.species()[j].thermo.h(T) - T * gas.species()[j].thermo.s(T)


In [12]:
# Get perturned G values

mod_gas = ct.Solution(cantera_file)
for j in range(N):
#     print(j)
    # change just the one reaction
    mod_gas.modify_species(j, perturbed_gas.species()[j])
    
    for i in range(temperatures.shape[1]):
    
        T = temperatures[0, i]
        mod_gas.TPX = T, pressures[0, i], phi_dicts[int(i / 51)]
        G_perturbed[j, i] = mod_gas.species()[j].thermo.h(T) - T * mod_gas.species()[j].thermo.s(T)

    mod_gas.modify_species(j, gas.species()[j])

For species Ar, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species He, discontinuity in h/RT detected at Tmid = 4762.74
	Value computed using low-temperature polynomial:  2.327935178063383
	Value computed using high-temperature polynomial: 2.320328165487312

For species butane(1), discontinuity in h/RT detected at Tmid = 1389.0
	Value computed using low-temperature polynomial:  6.672352745666894
	Value computed using high-temperature polynomial: 6.314788936675075

For species CH(3), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  81.27192833333334
	Value computed using high-temperature polynomial: 81.30542726833333

For species C2H(4), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  79.04515833333333
	Value computed using high-temperature 

For species C2H5O2(47), discontinuity in h/RT detected at Tmid = 1389.0
	Value computed using low-temperature polynomial:  9.956664416545813
	Value computed using high-temperature polynomial: 9.792960641216094

For species C2H6O2(48), discontinuity in h/RT detected at Tmid = 1390.0
	Value computed using low-temperature polynomial:  -1.9677150508723127
	Value computed using high-temperature polynomial: -2.1890157436045534

For species C2H5O(49), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  7.009500333333333
	Value computed using high-temperature polynomial: 6.85734259

For species C2H4O(51), discontinuity in h/RT detected at Tmid = 1000.0
	Value computed using low-temperature polynomial:  0.29742900000000283
	Value computed using high-temperature polynomial: 0.13550459666666725

For species C2H3O3(56), discontinuity in h/RT detected at Tmid = 1393.0
	Value computed using low-temperature polynomial:  3.7899779742663924
	Value computed

For species C4H7O(202), discontinuity in h/RT detected at Tmid = 1395.0
	Value computed using low-temperature polynomial:  22.488181349087316
	Value computed using high-temperature polynomial: 22.08819255065596

For species C4H8O(214), discontinuity in h/RT detected at Tmid = 1371.0
	Value computed using low-temperature polynomial:  6.69776039237299
	Value computed using high-temperature polynomial: 6.207492166569214

For species C4H9O(217), discontinuity in h/RT detected at Tmid = 1679.0
	Value computed using low-temperature polynomial:  15.845287841065236
	Value computed using high-temperature polynomial: 15.554039754599852

For species S(219), discontinuity in h/RT detected at Tmid = 1407.0
	Value computed using low-temperature polynomial:  19.962680157585844
	Value computed using high-temperature polynomial: 19.609072441877707

For species S(220), discontinuity in h/RT detected at Tmid = 1400.0
	Value computed using low-temperature polynomial:  21.28622322316191
	Value computed usi

For species C4H7(4343), discontinuity in h/RT detected at Tmid = 936.88
	Value computed using low-temperature polynomial:  43.23108488114937
	Value computed using high-temperature polynomial: 42.974074169338735

For species S(4407), discontinuity in h/RT detected at Tmid = 1493.99
	Value computed using low-temperature polynomial:  25.289221326036774
	Value computed using high-temperature polynomial: 25.170887720316664

For species C4H6(4880), discontinuity in h/RT detected at Tmid = 980.35
	Value computed using low-temperature polynomial:  72.68408493605675
	Value computed using high-temperature polynomial: 72.45145910529533

For species S(5093), discontinuity in h/RT detected at Tmid = 959.6
	Value computed using low-temperature polynomial:  -4.3547526077970495
	Value computed using high-temperature polynomial: -4.575863220776355

For species S(5151), discontinuity in h/RT detected at Tmid = 1562.93
	Value computed using low-temperature polynomial:  -0.6787876731554903
	Value computed

In [29]:
# G has units Enthalpy [J/kg or J/kmol] it's J / kmol
delta_G = G_perturbed - G_base
delta_G_kcal_mol = delta_G / 4.184 / 1000.0 / 1000.0  # needs to be kcal/mol to match Gao paper

### Now do reaction rate

In [ ]:
# except we know that by definition, this is 0.1

In [80]:
delta_k = 0.1 * np.ones((M, total_delays.shape[1]))

In [88]:
# concatenate into a big delta matrix
delta = np.concatenate((delta_G_kcal_mol, delta_k), axis=0)

In [114]:
# first derivative is change in delay / change in G
first_derivative = np.divide(d_ln_tau, delta)

In [117]:
# extract table 7, condition 7
index = 7 * 51 + 7
fd = first_derivative[:, index]

In [118]:
fd

matrix([[-0.00000000e+00],
        [-2.17350088e-08],
        [-0.00000000e+00],
        ...,
        [            nan],
        [            nan],
        [            nan]])

In [116]:
first_derivative[0]

matrix([[-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
         -0.00000000e+00, -0.00000000e+0

In [ ]:
def get_delta_G(ct_species_index):
    original 
    

In [ ]:
delta_ln_tau = np.log(total_delays) - np.log(total_base_delays)


# get the change in input parameter, either delta ln k or delta G
N = len(gas.species())
M = len(gas.reactions())

# delta_ln_k = 

